In [1]:
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_bloch_multivector, plot_state_qsphere
from random import getrandbits
import pickle

RETAINED_CHAR = '_'

In [2]:

class CryptoCircuit(QuantumCircuit):
    def __init__(self, size=7):
        # Each qubit representing a bit in ASCII (7-bit)
        super().__init__(size)

    def initialize(self, code:str):
        code = code[::-1] # reverse order
        for i in range(len(code)):
            self.x(i) if code[i]!='0' else self.i(i)

    def add_gate(self, gate:bool):
        self.h(self.qubits) if gate else self.i(self.qubits)

    def get_measurements(self):
        self.measure_all()
        sim = Aer.get_backend("qasm_simulator")
        counts = execute(self, sim, shots=1).result().get_counts()
        return list(counts)[0]

    def visualize(self):
        sim = Aer.get_backend("statevector_simulator")
        statevector = execute(self, sim, shots=1).result().get_statevector()
        display(plot_bloch_multivector(statevector))
        # display(plot_state_qsphere(statevector))
    
    def draw(self, output=""):
        return ("You are not allowed to print the circuit..")



In [3]:

class ProtocolBB84:

    def sender(message:str):
        circuits, gates = [], []
        for letter in message:
            # Skip the already found letters
            if letter == RETAINED_CHAR:
                circuits.append(None)
                gates.append(None)
                continue
        
            # Get letter's binary code
            letter = bin(ord(letter))[2:]

            # Create Quantum Circuit
            circuit = CryptoCircuit()

            # Alice's "random" byte
            circuit.initialize(letter)

            # Choose a random sending basis
            gates.append(getrandbits(1))
            circuit.add_gate(gates[-1])

            # Save circuit and gate
            circuits.append(circuit)

        return circuits, gates



In [4]:

message = "BB84 is a Quantum Key Distribution protocol relying on no-cloning theorem."

circuits_out, sender_gates = ProtocolBB84.sender(message)

# Send - write to file
with open("quantum_channel.txt", "bw") as f:
    for circuit in circuits_out:
        f.write( pickle.dumps(circuit) + b'\n\n')

with open("classical_channel.txt", "w") as f:
    for gate in sender_gates:
        f.write( str(gate) )

